# Arguments

In [1]:
# inceptionv3-ft-pv-self-eb,ap,lm,ct

num_of_epochs = 100
batch_size = 32
dataset_path_plantvillage = '../datasets/plantvillage'
dataset_path_self = '../datasets/self'
dataset_path_validation = '../datasets/validation'

dataset_paths = [dataset_path_plantvillage, dataset_path_self, dataset_path_validation]

final_model_path= 'model-final.h5'

input_width = 224
input_height = 224
input_depth = 3


# Select training classes

In [2]:
class_paths_validation = ['../datasets/validation/___Yellow_Leaf_Curl_Virus', '../datasets/validation/___Late_blight', '../datasets/validation/___Early_blight', '../datasets/validation/___Two-spotted_spider_mite', '../datasets/validation/___Septoria_leaf_spot', '../datasets/validation/___Bacterial_spot', '../datasets/validation/___healthy', '../datasets/validation/___Target_Spot', '../datasets/validation/___Mosaic_Virus', '../datasets/validation/___Leaf_Mold']

# Load dataset

In [3]:
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from keras.preprocessing.image   import ImageDataGenerator
from keras.optimizers import RMSprop
from keras.layers import Input
from keras.models import Model
from keras .applications import VGG16
from keras.optimizers import SGD
from keras.models import Model
from imutils import paths
import numpy as np
import os

import sys
sys.path.append('..')

from utils.preprocessors.resize_image_preprocessor import resizeImagePreprocessor
from utils.preprocessors.img_to_array_preprocessor import ImgToArrayPreprocessor
from utils.io.dataset_loader import DatasetLoader

# construct the image generator for data augmentation
aug = ImageDataGenerator(   rotation_range=30,
                            width_shift_range=0.1,
                            height_shift_range=0.1,
                            shear_range=0.2,
                            zoom_range=0.2,
                            horizontal_flip=True,
                            fill_mode='nearest')

# Load image paths
image_paths = []
print("[INFO] loading images...")
for path in class_paths_validation:
    image_paths.extend(list(paths.list_images(path)))

# Get unique classnames
class_names = [pt.split(os.path.sep)[-2] for pt in image_paths]
class_names = [str(x) for x in np.unique(class_names)]

# Initial image preprocessing
aap = resizeImagePreprocessor(input_width, input_height)
iap= ImgToArrayPreprocessor()

#Load image data and perform image data preprocessing
dl = DatasetLoader(preprocessors=[aap,iap])
(test_x,test_y)  = dl.load(image_paths,verbose=500)
test_x = test_x.astype("float") / 255.0

# convert the labels from integers to vectors
test_y = LabelBinarizer().fit_transform(test_y)



Using TensorFlow backend.


[INFO] loading images...


In [4]:
print(len(class_names))
print(class_names)

10
['___Bacterial_spot', '___Early_blight', '___Late_blight', '___Leaf_Mold', '___Mosaic_Virus', '___Septoria_leaf_spot', '___Target_Spot', '___Two-spotted_spider_mite', '___Yellow_Leaf_Curl_Virus', '___healthy']


# Load model from disk

In [5]:
# import tensorflow as tf
# print(tf.__version__)


# model = tf.keras.models.load_model(
#     final_model_path,
#     custom_objects=None,
#     compile=False
# )

from keras.models import load_model

model = load_model(final_model_path)

# Evaluate

In [6]:
from sklearn.metrics import classification_report

print("[INFO] evaluating after initialization...")
predictions = model.predict(test_x,batch_size=batch_size)

print(classification_report(test_y.argmax(axis =1),
                            predictions.argmax(axis =1),
                            target_names=class_names, 
                            digits=4))

[INFO] evaluating after initialization...
                            precision    recall  f1-score   support

         ___Bacterial_spot     0.4000    0.0308    0.0571        65
           ___Early_blight     0.3488    0.5357    0.4225        56
            ___Late_blight     0.4065    0.8475    0.5495       118
              ___Leaf_Mold     0.0000    0.0000    0.0000       106
           ___Mosaic_Virus     0.0000    0.0000    0.0000        20
     ___Septoria_leaf_spot     0.0476    0.5714    0.0879         7
            ___Target_Spot     0.0000    0.0000    0.0000        24
___Two-spotted_spider_mite     0.0000    0.0000    0.0000         4
 ___Yellow_Leaf_Curl_Virus     0.1000    0.0500    0.0667        20
                ___healthy     0.0000    0.0000    0.0000        19

               avg / total     0.2183    0.3121    0.2145       439



/usr/local/lib/python3.5/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
